# BAML Logging Demo - Testing Log Capture in Notebooks

This notebook demonstrates how BAML output is captured in Jupyter notebooks.

In [ ]:
!pip install baml-py==0.202.0 pydantic

In [ ]:
import subprocess
import os
import sys
from IPython.utils.capture import capture_output

# Set up environment
if 'OPENAI_API_KEY' not in os.environ:
    print("Warning: OPENAI_API_KEY not set")

# Set BAML logging
os.environ['BAML_LOG'] = 'info'

def baml_generate():
    try:
        result = subprocess.run(
            ["baml-cli", "generate"],
            check=True,
            capture_output=True,
            text=True
        )
        if result.stdout:
            print("[baml-cli generate]\n", result.stdout)
        if result.stderr:
            print("[baml-cli generate stderr]\n", result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"baml-cli generate failed: {e}")
        raise

def get_baml_client():
    baml_generate()
    import sys
    modules_to_delete = [key for key in sys.modules.keys() if key.startswith('baml_client')]
    for module in modules_to_delete:
        del sys.modules[module]
    import baml_client
    return baml_client.sync_client.b

In [ ]:
# Initialize BAML
!baml-cli init

In [ ]:
# Create a simple BAML file
baml_content = '''class DoneForNow {
  intent "done_for_now"
  message string
}

function DetermineNextStep(thread string) -> DoneForNow {
  client OpenAI/gpt-4o-mini
  prompt #"
    Given the conversation thread, determine the next step.
    
    Thread:
    {{ thread }}
    
    Respond with a message.
  "#
}
'''

with open('baml_src/agent.baml', 'w') as f:
    f.write(baml_content)
    
print("Created agent.baml")

In [ ]:
# Helper function to capture BAML logs
def run_with_baml_logs(func, *args, **kwargs):
    """Run a function and capture BAML logs in the notebook output."""
    print(f"Running with BAML_LOG={os.environ.get('BAML_LOG')}...")
    
    # Capture both stdout and stderr
    with capture_output() as captured:
        result = func(*args, **kwargs)
    
    # Display the result first
    if result is not None:
        print("=== Result ===")
        print(result)
    
    # Display captured stdout if any
    if captured.stdout:
        print("\n=== Stdout ===")
        print(captured.stdout)
    
    # Display BAML logs from stderr
    if captured.stderr:
        print("\n=== BAML Logs (from stderr) ===")
        print(captured.stderr)
    
    return result

In [ ]:
# Test function that uses BAML
def test_baml_call():
    b = get_baml_client()
    thread = '[{"type": "user_input", "data": "Hello, how are you?"}]'
    result = b.DetermineNextStep(thread)
    return result

In [ ]:
# Run without log capture
print("=== Running WITHOUT log capture ===")
result1 = test_baml_call()
print(f"Result: {result1}")

In [ ]:
# Run WITH log capture
print("=== Running WITH log capture ===")
result2 = run_with_baml_logs(test_baml_call)

In [ ]:
# Test with different log levels
print("\n=== Testing with DEBUG log level ===")
os.environ['BAML_LOG'] = 'debug'
result3 = run_with_baml_logs(test_baml_call)

## Summary

This notebook demonstrates:
1. BAML logs are written to stderr by default
2. Using `capture_output()` from IPython can capture these logs
3. The `run_with_baml_logs()` helper function makes it easy to see BAML logs in notebooks
4. Different log levels (info, debug) show different amounts of detail